# Transformer From Scratch

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pdb, math
from copy import deepcopy

import numpy as np
np.set_printoptions(precision=4)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context(context="talk")
%matplotlib inline

import torch;
assert(torch.cuda.is_available())
from torch import nn
from torch.nn import functional as F

In [3]:
from transformer import *

AttributeError: module 'transformer.transformer' has no attribute 'PositionWiseFNN'

In [ ]:
d_model = 10
bs = 3
seq_len = 4
n_heads = 5
d_ff = 40

dropout = 0.5
valid_len = torch.tensor([2,3,2])

x = torch.rand(bs, seq_len, d_model)
# x = torch.ones(bs, seq_len, d_model)
query,key,value = x,x,x